In [0]:
import numpy as np
import torch
from torch import FloatTensor
import torch.nn.functional as f

np.random.seed(598020)

y = np.random.randn(2,1)
x = np.random.randn(2,1)
w1 = np.random.randn(3,2)
w2 = np.random.randn(3,3)
w3 = np.random.randn(2,3)

In [0]:
relu = lambda x : np.maximum(x,0)

In [0]:
def relu_grad(op):
  op[op<0] = 0
  op[op>0] = 1
  return op

In [0]:
v1 = np.dot(w1,x)
v2 = relu(v1)
v3 = np.dot(w2,v2)
v4 = relu(v3)
v5 = np.dot(w3,v4)
D = (y-v5)
loss = 0.5*np.sum(np.dot(D.T, D))

d_w3_a = -np.dot(D, v4.T)
d_w2_a = -np.dot(np.multiply(np.dot(w3.T, D), relu_grad(v3)), v2.T)
d_w1_a = -np.dot(np.multiply(np.dot(w2.T, np.multiply(np.dot(w3.T, D), relu_grad(v3))), relu_grad(v1)), x.T)

# D_grad = D
v5_d_r = -D
v4_d_r = np.dot(w3.T, v5_d_r)
w3_d_r = np.dot(v5_d_r, v4.T)
v3_d_r = np.multiply(v4_d_r, relu_grad(v3))
w2_d_r = np.dot(v3_d_r, v2.T)
v2_d_r = np.dot(w2.T, v3_d_r)
v1_d_r = np.multiply(v2_d_r, relu_grad(v1))
w1_d_r = np.dot(v1_d_r, x.T)

In [0]:
x = torch.from_numpy(x)
y = torch.from_numpy(y)
w1 = torch.from_numpy(w1)
w1.requires_grad = True
w2 = torch.from_numpy(w2)
w2.requires_grad = True
w3 = torch.from_numpy(w3)
w3.requires_grad = True

v1 = torch.mm(w1, x)
v2 = f.relu(v1)
v3 = torch.mm(w2, v2)
v4 = f.relu(v3)
v5 = torch.mm(w3, v4)
D = (y-v5)
loss = 0.5*torch.mm(D.T, D)
loss.backward()

In [22]:
print("Analytical Gradient values: \n w1 = ",d_w1_a, "\n w2 = ",d_w2_a, "\n w3 = ",d_w3_a)

Analytical Gradient values: 
 w1 =  [[20.21253439  7.85019704]
 [-0.         -0.        ]
 [ 7.39915332  2.87370254]] 
 w2 =  [[-3.24126614 -0.         -6.52755207]
 [15.72038528 -0.         31.65911995]
 [-3.13005205 -0.         -6.30357918]] 
 w3 =  [[ 3.72117366 15.45896671 15.41956259]
 [ 2.19570832  9.12168714  9.09843642]]


In [23]:
print("Reverse-mode autodiff Gradient values :\n w1 = ",w1_d_r, "\n w2 = ",w2_d_r, "\n w3 = ",w3_d_r)

Reverse-mode autodiff Gradient values :
 w1 =  [[20.21253439  7.85019704]
 [ 0.          0.        ]
 [ 7.39915332  2.87370254]] 
 w2 =  [[-3.24126614  0.         -6.52755207]
 [15.72038528  0.         31.65911995]
 [-3.13005205  0.         -6.30357918]] 
 w3 =  [[ 3.72117366 15.45896671 15.41956259]
 [ 2.19570832  9.12168714  9.09843642]]


In [24]:
print("Torch Gradient Values: \n w1 = \n",w1.grad.numpy(),"\n w2 = ",w2.grad.numpy(),"\n w3 = ",w3.grad.numpy())

Torch Gradient Values: 
 w1 = 
 [[20.21253439  7.85019704]
 [ 0.          0.        ]
 [ 7.39915332  2.87370254]] 
 w2 =  [[-3.24126614 -0.         -6.52755207]
 [15.72038528  0.         31.65911995]
 [-3.13005205 -0.         -6.30357918]] 
 w3 =  [[ 3.72117366 15.45896671 15.41956259]
 [ 2.19570832  9.12168714  9.09843642]]
